In [1064]:
import os
import pandas as pd

In [1065]:
df = pd.read_json(os.path.join("data", "ceske_lvy_raw.json"))

## Nechybí žádný rok?

In [1066]:
df.groupby('rok').size()

rok
1993     10
1994     79
1995     46
1996     54
1997     47
1998     54
1999     53
2000     53
2001     56
2002     59
2003     51
2004     48
2005     46
2006     54
2007     53
2008     46
2009     53
2010     92
2011    110
2012    121
2013     89
2014     95
2015    102
2016    117
2017    115
2018    120
2019    108
2020    143
2021    148
2022    155
2023    142
dtype: int64

In [1067]:
len(df.groupby('rok').size())

31

In [1068]:
len(df)

2519

## Data na data

In [1069]:
df['datum'] = pd.to_datetime(df['datum'])

In [1070]:
df

,index,rok,datum,kategorie,kdo,film,co
0,0,1993,1994-02-25,Nejlepší film,Jiří Ježek (132937),Šakalí léta,vyhra
1,1,1993,1994-02-25,Nejlepší režie,Jan Hřebejk (3195),Šakalí léta,vyhra
2,2,1993,1994-02-25,Hlavní ženský herecký výkon,Jiřina Bohdalová (1088),Nesmrtelná teta,vyhra
3,3,1993,1994-02-25,Hlavní mužský herecký výkon,Josef Abrhám (931),Šakalí léta,vyhra
4,4,1993,1994-02-25,Nejlepší scénář,Václav Šašek (40793),Helimadoe,vyhra
...,...,...,...,...,...,...,...
2514,137,2023,2024-03-09,Cena Magnesia za nejlepší studentský film,Anna Wowra (249417),Jsme v tom spolu,nominace
2515,138,2023,2024-03-09,Cena Magnesia za nejlepší studentský film,Petr Pylypčuk (578198),Osmý den,nominace
2516,139,2023,2024-03-09,Nejlepší filmový plakát,Zuzana Burgrová (/cz/detail?creator=Zuzana Bur...,Němá tajemství,vyhra
2517,140,2023,2024-03-09,Nejlepší filmový plakát,Robert V. Novák (/cz/detail?creator=Robert V. ...,Němá tajemství,vyhra


## A teď správné filmy a persony

### Je tam nějaká shoda jmen?

In [1071]:
df['id'] = df['kdo'].apply(lambda x: x.split("(")[-1].replace(")",""))

In [1072]:
df['jmeno'] = df['kdo'].apply(lambda x: x.split("(")[0].strip())

In [1073]:
df.groupby('jmeno')['id'].nunique().nlargest(20)

jmeno
Alice Nellis                    2
Deana Jakubisková-Horváthová    2
Ivo Špalj                       2
Jan Svěrák                      2
Jan Švankmajer                  2
Jarosław Kamiński               2
Jiří Svoboda                    2
Martha Issová                   2
Martin Ryšavý                   2
Michal Holubec                  2
Ondřej Trojan                   2
Petr Horký                      2
Radim Procházka                 2
Radim Špaček                    2
Robert Sedláček                 2
Vladimír Merta                  2
Vít Klusák                      2
Čestmír Kopecký                 2
AQS a.s.                        1
Adam Brothánek                  1
Name: id, dtype: int64

In [1074]:
duplikantstvo = df.groupby('jmeno')['id'].nunique().nlargest(18).index.to_list()

In [1075]:
for d in duplikantstvo:
    print(d.upper())
    print(df[df['jmeno'] == d][['kategorie','rok','film']])

ALICE NELLIS
                   kategorie   rok             film
389   Cena filmových kritiků  2000         Ene bene
457           Nejlepší režie  2002            Výlet
471          Nejlepší scénář  2002            Výlet
505   Cena filmových kritiků  2002            Výlet
715           Nejlepší režie  2007         Tajnosti
729          Nejlepší scénář  2007         Tajnosti
759   Cena filmových kritiků  2007         Tajnosti
903          Nejlepší scénář  2010    Mamas & Papas
1197          Nejlepší režie  2013          Revival
1226         Nejlepší scénář  2013          Revival
1381          Nejlepší režie  2015  Sedmero krkavců
1419         Nejlepší scénář  2015  Sedmero krkavců
2131         Nejlepší scénář  2021          Zátopek
2428         Nejlepší scénář  2023   Němá tajemství
DEANA JAKUBISKOVÁ-HORVÁTHOVÁ
                  kategorie   rok                          film
235  Cena filmových kritiků  1997  Nejasná zpráva o konci světa
343           Nejlepší film  2000                 

Podle všeho tam je dvakrát Jiří Svoboda.

### Persony

In [1076]:
originalni_jmena = df['kdo'].drop_duplicates().to_list()

In [1077]:
persony = pd.read_json(os.path.join("data","persony.json"))

In [1078]:
persony

,Jméno,Id,Medailon,Osoba,Alternativní jména,Životopis,Datum narození,Místo narození,Datum úmrtí,Místo úmrtí,Město narození,Stát narození,Město úmrtí,Stát úmrtí,Rok narození,Rok úmrtí
0,Ondřej Mašek (1000),1000,,fyzická,None,None,None,None,None,None,None,None,None,None,NaN,NaN
1,kino Světozor (100019),100019,None,právnická,None,None,None,None,None,None,None,None,None,None,NaN,NaN
2,"AD WOOD, s.r.o. (100038)",100038,None,právnická,None,None,None,None,None,None,None,None,None,None,NaN,NaN
3,Hypermarket Film s.r.o. (100039),100039,None,právnická,Hypermarket Film,None,None,None,None,None,None,None,None,None,NaN,NaN
4,Miloš Novotný (100040),100040,,fyzická,None,None,None,None,None,None,None,None,None,None,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33128,Josef Dvořák (9998),9998,"herec, divadelní podnikatel",fyzická,Josef Voloďa Dvořák (rodné jméno),None,1942-04-25T00:00:00.000,"Horní Cerekev, Československo",None,None,Horní Cerekev,Československo,None,None,1942.0,NaN
33129,Antonín Dimitrov (99996),99996,"výtvarník, grafik, scénograf, designér",fyzická,None,None,1928-02-27T00:00:00.000,"Mšecké Žehrovice, Československo",2014-12-27T00:00:00.000,"Bobcaygeon, Kanada",Mšecké Žehrovice,Československo,Bobcaygeon,Kanada,1928.0,2014.0
33130,Čechomor (person),person,None,právnická,None,None,None,None,None,None,None,None,None,None,NaN,NaN
33131,Kevin Mc Kidd (person),person,,fyzická,None,None,None,None,None,None,None,None,None,None,NaN,NaN


In [1079]:
persony['kdo'] = persony['Jméno'].apply(lambda x: x.split("(")[0].strip())

In [1080]:
lvovstvo = df['jmeno'].drop_duplicates().to_list()

In [1081]:
persony[persony['kdo'].isin(lvovstvo)].groupby('kdo')['Jméno'].nunique().nlargest(60)

kdo
Jan Novák            6
Jiří Svoboda         6
Martin Čech          5
Tomáš Svoboda        5
Martin Dušek         4
Filip Marek          3
Jan Novotný          3
Jan Němec            3
Jiří Černý           3
Josef Jelínek        3
Karel Černý          3
Lukáš Král           3
Petr Král            3
Petr Slavík          3
Richard Němec        3
Václav Novák         3
Adam Novák           2
Boris Masník         2
Eduard Kučera        2
Eva Pavlíčková       2
Ivan Horák           2
Ivana Kačírková      2
Jan Bílek            2
Jan Hartl            2
Jan Kraus            2
Jan Vlasák           2
Jan Vlček            2
Jan Šťastný          2
Jana Vlčková         2
Jaroslav Bouček      2
Jaroslav Kučera      2
Jiří Havelka         2
Jiří Hubač           2
Jiří Sternwald       2
Jiří Strach          2
Josef Trojan         2
Karel Havlíček       2
Karel Vacek          2
Karl Baumgartner     2
Michal Dvořák        2
Michal Čech          2
Miroslav Hanuš       2
Ondřej Ježek         2
Ondřej 

In [1082]:
# klíč = akademie, value = přehled
# Jiří Svoboda pořešit
# Jiří Strach
# Ondřej Moravec
#    'Pavel Soukup (74494)': 
# chybí např. zátopek
sparovani = {
    'Jan Novák (51626)': 'Jan Novák (19815)',
    'Martin Čech (82865)': 'Martin Čech (6316)',
    'Tomáš Svoboda (242580)': 'Tomáš Svoboda (21548)',
    'Martin Dušek (9803)': 'Martin Dušek (19085)',
    'Filip Marek (151607)': 'Filip Marek (83788)',
    'Jan Novotný (107289)': 'Jan Novotný (22567)',
    'Jan Němec (3244)': 'Jan Němec (3041)',
    'Jiří Černý (92711)': 'Jiří Černý (13756)',
    'Josef Jelínek (138874)': 'Josef Jelínek (3872)',
    'Karel Černý (1637)': 'Karel Černý (1241)',
    'Lukáš Král (107295)': 'Lukáš Král (18937)',
    'Petr Král (274701)': 'Petr Král (3735)',
    'Petr Slavík (15065)': 'Petr Slavík (487)',
    'Richard Němec (104140)': 'Richard Němec (633)',
    'Václav Novák (107285)': 'Václav Novák (18107)',
    'Adam Novák (22917)': 'Adam Novák (24432)',
    'Boris Masník (86416)': 'Boris Masník (1489)',
    'Eduard Kučera (/cz/detail?creator=Eduard Kučera)': 'Eduard Kučera (78741)',
    'Eva Pavlíčková (530029)': 'Eva Pavlíčková (33413)',
    'Ivan Horák (137493)': 'Ivan Horák (2298)',
    'Ivana Kačírková (132917)': 'Ivana Kačírková (2635)',
    'Jan Bílek (136618)': 'Jan Bílek (4429)',
    'Jan Hartl (1058)': 'Jan Hartl (7297)',
    'Jan Kraus (1517)': 'Jan Kraus (1062)',
    'Jan Vlasák (140082)': 'Jan Vlasák (1805)',
    'Jan Vlček (138900)': 'Jan Vlček (3233)',
    'Jan Šťastný (181596)': 'Jan Šťastný (2439)',
    'Jana Vlčková (137599)': 'Jana Vlčková (19087)',
    'Jaroslav Bouček (70896)': 'Jaroslav Bouček (146)',
    'Jaroslav Kučera (137263)': 'Jaroslav Kučera (1647)',
    'Jiří Havelka (54648)': 'Jiří Havelka (28977)',
    'Jiří Hubač (37327)': 'Jiří Hubač (848)',
    'Jiří Sternwald (140013)': 'Jiří Sternwald (831)',
    'Josef Trojan (233076)': 'Josef Trojan (83672)',
    'Karel Havlíček (71504)': 'Karel Havlíček (26509)',
    'Karel Vacek (140240)': 'Karel Vacek (2824)',
    'Karl Baumgartner (148878)': 'Karl Baumgartner (21976)',
    'Michal Dvořák (48388)': 'Michal Dvořák (8266)',
    'Michal Čech (137494)': 'Michal Čech (2347)',
    'Miroslav Hanuš (54461)': 'Miroslav Hanuš (3267)',
    'Ondřej Ježek (76196)': 'Ondřej Ježek (1554)',
    'Ondřej Malý (55912)': 'Ondřej Malý (59148)',
    'Pavel Hrdlička (107169)': 'Pavel Hrdlička (5445)',
    'Pavel Liška (1529)': 'Pavel Liška (984)',
    'Pavel Nový (1062)': 'Pavel Nový (510)',
    'Pavel Vácha (55233)': 'Pavel Vácha (24312)',
    'Petr Bílek (137899)': 'Petr Bílek (1606)',
    'Petr Horký (35668)': 'Petr Horký (3205)',
    'Petr Horký (342416)': 'Petr Horký (3205)',
    'Petr Pýcha (383402)': 'Petr Pýcha (401774)',
    'Petr Štěpán (33351)': 'Petr Štěpán (5407)',
    'Richard Müller (107165)': 'Richard Müller (9113)',
    'Tomáš Bělohradský (137618)': 'Tomáš Bělohradský (26932)',
    'Václav Neužil (55565)': 'Václav Neužil (9101)',
    'Zuzana Stivínová (1335)': 'Zuzana Stivínová (3511)'
}

In [1083]:
for key, val in sparovani.items():
    df['kdo'] = df['kdo'].replace(key, val)

In [1084]:
df.loc[(df['kategorie'] == 'Nejlepší hudba') & (df['kdo'] == 'Jiří Svoboda (65531)'), 'kdo'] = 'Jiří Svoboda (21486)'

In [1085]:
df.loc[(df['kategorie'] == 'Nejlepší televizní film nebo minisérie') & (df['kdo'] == 'Jiří Svoboda (3297)'), 'kdo'] = 'Jiří Svoboda (1703)'

In [1086]:
opraveni = persony[persony['kdo'].isin(lvovstvo)].groupby('kdo')['Jméno'].nunique().nlargest(58).index.to_list()
opraveni

['Jan Novák',
 'Jiří Svoboda',
 'Martin Čech',
 'Tomáš Svoboda',
 'Martin Dušek',
 'Filip Marek',
 'Jan Novotný',
 'Jan Němec',
 'Jiří Černý',
 'Josef Jelínek',
 'Karel Černý',
 'Lukáš Král',
 'Petr Král',
 'Petr Slavík',
 'Richard Němec',
 'Václav Novák',
 'Adam Novák',
 'Boris Masník',
 'Eduard Kučera',
 'Eva Pavlíčková',
 'Ivan Horák',
 'Ivana Kačírková',
 'Jan Bílek',
 'Jan Hartl',
 'Jan Kraus',
 'Jan Vlasák',
 'Jan Vlček',
 'Jan Šťastný',
 'Jana Vlčková',
 'Jaroslav Bouček',
 'Jaroslav Kučera',
 'Jiří Havelka',
 'Jiří Hubač',
 'Jiří Sternwald',
 'Jiří Strach',
 'Josef Trojan',
 'Karel Havlíček',
 'Karel Vacek',
 'Karl Baumgartner',
 'Michal Dvořák',
 'Michal Čech',
 'Miroslav Hanuš',
 'Ondřej Ježek',
 'Ondřej Malý',
 'Ondřej Moravec',
 'Pavel Hrdlička',
 'Pavel Liška',
 'Pavel Nový',
 'Pavel Soukup',
 'Pavel Vácha',
 'Petr Bílek',
 'Petr Horký',
 'Petr Pýcha',
 'Petr Štěpán',
 'Richard Müller',
 'Tomáš Bělohradský',
 'Václav Neužil',
 'Zuzana Stivínová']

In [1087]:
persony['jmeno_bez_id'] = persony['Jméno'].apply(lambda x: x.split("(")[0].strip())

In [1088]:
persony[['jmeno_bez_id','Id']]

,jmeno_bez_id,Id
0,Ondřej Mašek,1000
1,kino Světozor,100019
2,"AD WOOD, s.r.o.",100038
3,Hypermarket Film s.r.o.,100039
4,Miloš Novotný,100040
...,...,...
33128,Josef Dvořák,9998
33129,Antonín Dimitrov,99996
33130,Čechomor,person
33131,Kevin Mc Kidd,person


In [1089]:
opravena_id = [x.split("(")[1].replace(")","") for x in df[df['jmeno'].isin(opraveni)]['kdo'].to_list()]

In [1090]:
opravena_id

['8266',
 '21486',
 '8266',
 '1489',
 '7297',
 '848',
 '848',
 '21976',
 '2635',
 '8266',
 '2635',
 '19815',
 '2824',
 '633',
 '146',
 '4429',
 '3511',
 '2635',
 '831',
 '633',
 '146',
 '4429',
 '984',
 '848',
 '6316',
 '2824',
 '3511',
 '146',
 '1647',
 '984',
 '848',
 '1805',
 '1062',
 '3872',
 '2824',
 '3511',
 '1805',
 '5445',
 '3041',
 '984',
 '3041',
 '3735',
 '3041',
 '5445',
 '26932',
 '146',
 '5407',
 '984',
 '1805',
 '24432',
 '5445',
 '59148',
 '26932',
 '146',
 '1554',
 '1554',
 '22567',
 '5445',
 '2347',
 '1805',
 '18937',
 '18937',
 '1805',
 '1241',
 '13756',
 '5445',
 '9113',
 '18107',
 '22567',
 '18937',
 '19085',
 '3205',
 '2298',
 '3233',
 '1805',
 '18937',
 '2439',
 '2347',
 '22567',
 '18937',
 '74494',
 '3267',
 '9113',
 '18937',
 '510',
 '9101',
 '19087',
 '1805',
 '1805',
 '18937',
 '146',
 '3205',
 '7297',
 '401774',
 '19087',
 '21548',
 '831',
 '19085',
 '28977',
 '19085',
 '28977',
 '19087',
 '9113',
 '1805',
 '83672',
 '2439',
 '5445',
 '22567',
 '18937',
 '91

In [1091]:
persony = persony[~(persony['jmeno_bez_id'].isin(opraveni) & (~persony['Id'].isin(opravena_id)))]

In [1092]:
persony

,Jméno,Id,Medailon,Osoba,Alternativní jména,Životopis,Datum narození,Místo narození,Datum úmrtí,Místo úmrtí,Město narození,Stát narození,Město úmrtí,Stát úmrtí,Rok narození,Rok úmrtí,kdo,jmeno_bez_id
0,Ondřej Mašek (1000),1000,,fyzická,None,None,None,None,None,None,None,None,None,None,NaN,NaN,Ondřej Mašek,Ondřej Mašek
1,kino Světozor (100019),100019,None,právnická,None,None,None,None,None,None,None,None,None,None,NaN,NaN,kino Světozor,kino Světozor
2,"AD WOOD, s.r.o. (100038)",100038,None,právnická,None,None,None,None,None,None,None,None,None,None,NaN,NaN,"AD WOOD, s.r.o.","AD WOOD, s.r.o."
3,Hypermarket Film s.r.o. (100039),100039,None,právnická,Hypermarket Film,None,None,None,None,None,None,None,None,None,NaN,NaN,Hypermarket Film s.r.o.,Hypermarket Film s.r.o.
4,Miloš Novotný (100040),100040,,fyzická,None,None,None,None,None,None,None,None,None,None,NaN,NaN,Miloš Novotný,Miloš Novotný
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33128,Josef Dvořák (9998),9998,"herec, divadelní podnikatel",fyzická,Josef Voloďa Dvořák (rodné jméno),None,1942-04-25T00:00:00.000,"Horní Cerekev, Československo",None,None,Horní Cerekev,Československo,None,None,1942.0,NaN,Josef Dvořák,Josef Dvořák
33129,Antonín Dimitrov (99996),99996,"výtvarník, grafik, scénograf, designér",fyzická,None,None,1928-02-27T00:00:00.000,"Mšecké Žehrovice, Československo",2014-12-27T00:00:00.000,"Bobcaygeon, Kanada",Mšecké Žehrovice,Československo,Bobcaygeon,Kanada,1928.0,2014.0,Antonín Dimitrov,Antonín Dimitrov
33130,Čechomor (person),person,None,právnická,None,None,None,None,None,None,None,None,None,None,NaN,NaN,Čechomor,Čechomor
33131,Kevin Mc Kidd (person),person,,fyzická,None,None,None,None,None,None,None,None,None,None,NaN,NaN,Kevin Mc Kidd,Kevin Mc Kidd


In [1093]:
df = df.merge(persony[['jmeno_bez_id','Id','Datum narození','Datum úmrtí']],left_on='jmeno',right_on='jmeno_bez_id', how='outer')

In [1094]:
df

,index,rok,datum,kategorie,kdo,film,co,id,jmeno,jmeno_bez_id,Id,Datum narození,Datum úmrtí
0,0.0,1993.0,1994-02-25,Nejlepší film,Jiří Ježek (132937),Šakalí léta,vyhra,132937,Jiří Ježek,Jiří Ježek,4473,1950-06-01T00:00:00.000,None
1,0.0,1997.0,1998-02-28,Nejlepší film,Jiří Ježek (132937),Báječná léta pod psa,nominace,132937,Jiří Ježek,Jiří Ježek,4473,1950-06-01T00:00:00.000,None
2,1.0,1993.0,1994-02-25,Nejlepší režie,Jan Hřebejk (3195),Šakalí léta,vyhra,3195,Jan Hřebejk,Jan Hřebejk,4831,1967-06-27T00:00:00.000,None
3,7.0,1999.0,2000-03-04,Nejlepší režie,Jan Hřebejk (3195),Pelíšky,nominace,3195,Jan Hřebejk,Jan Hřebejk,4831,1967-06-27T00:00:00.000,None
4,51.0,1999.0,2000-03-04,Cena filmových kritiků,Jan Hřebejk (3195),Pelíšky,nominace,3195,Jan Hřebejk,Jan Hřebejk,4831,1967-06-27T00:00:00.000,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
34702,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,Alena Sasínová,9995,None,None
34703,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,plutoon s. r. o.,99962,None,None
34704,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,Bedřich Karen,99963,1887-10-15T00:00:00.000,1964-08-21T00:00:00.000
34705,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,Jožka Schneider,99965,1892-03-14T00:00:00.000,1960-01-27T00:00:00.000


In [1095]:
df = df[~df['rok'].isnull()]

In [1096]:
len(df)

2519

In [1097]:
def dopln_id_fp(radek):
    if radek['jmeno_bez_id'] != None:
        if radek['jmeno'] not in opraveni:
            return f"""{radek['jmeno_bez_id']} ({radek['Id']})"""
        else:
            return radek['kdo']
    else:
        return radek['kdo']

In [1098]:
df['kdo'] = df.apply(lambda row: dopln_id_fp(row), axis=1)

C:\Users\micha\AppData\Local\Temp\ipykernel_25248\3399992892.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['kdo'] = df.apply(lambda row: dopln_id_fp(row), axis=1)


In [1099]:
df

,index,rok,datum,kategorie,kdo,film,co,id,jmeno,jmeno_bez_id,Id,Datum narození,Datum úmrtí
0,0.0,1993.0,1994-02-25,Nejlepší film,Jiří Ježek (4473),Šakalí léta,vyhra,132937,Jiří Ježek,Jiří Ježek,4473,1950-06-01T00:00:00.000,None
1,0.0,1997.0,1998-02-28,Nejlepší film,Jiří Ježek (4473),Báječná léta pod psa,nominace,132937,Jiří Ježek,Jiří Ježek,4473,1950-06-01T00:00:00.000,None
2,1.0,1993.0,1994-02-25,Nejlepší režie,Jan Hřebejk (4831),Šakalí léta,vyhra,3195,Jan Hřebejk,Jan Hřebejk,4831,1967-06-27T00:00:00.000,None
3,7.0,1999.0,2000-03-04,Nejlepší režie,Jan Hřebejk (4831),Pelíšky,nominace,3195,Jan Hřebejk,Jan Hřebejk,4831,1967-06-27T00:00:00.000,None
4,51.0,1999.0,2000-03-04,Cena filmových kritiků,Jan Hřebejk (4831),Pelíšky,nominace,3195,Jan Hřebejk,Jan Hřebejk,4831,1967-06-27T00:00:00.000,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2514,132.0,2023.0,2024-03-09,Nejlepší krátký film,nan (nan),Osmý den,nominace,422276,Tomáš Pertold,NaN,NaN,NaN,NaN
2515,133.0,2023.0,2024-03-09,Nejlepší krátký film,nan (nan),Osmý den,nominace,578198,Petr Pylypčuk,NaN,NaN,NaN,NaN
2516,138.0,2023.0,2024-03-09,Cena Magnesia za nejlepší studentský film,nan (nan),Osmý den,nominace,578198,Petr Pylypčuk,NaN,NaN,NaN,NaN
2517,135.0,2023.0,2024-03-09,Cena Magnesia za nejlepší studentský film,nan (nan),Doprovázení,nominace,616601,Vojtěch Konečný,NaN,NaN,NaN,NaN


In [1100]:
df[df['jmeno'] == 'Martin Čech']

,index,rok,datum,kategorie,kdo,film,co,id,jmeno,jmeno_bez_id,Id,Datum narození,Datum úmrtí
763,25.0,1999.0,2000-03-04,Nejlepší kamera,Martin Čech (6316),Kuře melancholik,vyhra,82865,Martin Čech,Martin Čech,6316,1962-06-25T00:00:00.000,None


### Filmy

In [1101]:
len(df)

2519

In [1102]:
df.groupby('film')['rok'].nunique().nlargest()

film
                                                  30
Atestace                                           2
#annaismissing                                     1
...a bude hůř                                      1
25 ze šedesátých aneb Československá nová vlna     1
Name: rok, dtype: int64

In [1103]:
df[df['film'] == 'Atestace']

,index,rok,datum,kategorie,kdo,film,co,id,jmeno,jmeno_bez_id,Id,Datum narození,Datum úmrtí
2442,148.0,2022.0,2023-03-04,Cena Magnesia za nejlepší studentský film,Jan Hecht (86597),Atestace,nominace,89699,Jan Hecht,Jan Hecht,86597,1993-02-04T00:00:00.000,None
2443,128.0,2023.0,2024-03-09,Nejlepší krátký film,Jan Hecht (86597),Atestace,nominace,89699,Jan Hecht,Jan Hecht,86597,1993-02-04T00:00:00.000,None
2509,127.0,2023.0,2024-03-09,Nejlepší krátký film,nan (nan),Atestace,nominace,792017,Marek Dusil,NaN,NaN,NaN,NaN


In [1104]:
filmy = pd.read_json(os.path.join("data", "filmy.json"))

In [1105]:
filmy['film_bez_id'] = filmy['Film'].apply(lambda x: x.split("(")[0].strip())

In [1106]:
len(df)

2519

In [1107]:
df['film_bez_id'] = df['film']

C:\Users\micha\AppData\Local\Temp\ipykernel_25248\340162799.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['film_bez_id'] = df['film']


In [1108]:
len(df)

2519

In [1109]:
filmy

,Film,Země původu,Copyright,Rok výroby,Premiéra,Minutáž,Režie,Kategorie,Žánr,Typologie,...,Vývozní společnost,Umělecké vedení,Zpěv role,Překlad české verze,Rychlost,Přístupnost,cyklus,Začátek výroby,Konec výroby,film_bez_id
0,Staroměstští hasiči (401702),Rakousko-Uhersko,1898,1898,premiéra 28. 8. 1898 (Výstava architektury a ...,1.0,[Jan Kříženecký (127419)],film,reportáž,"[dokumentární, distribuční]",...,None,None,None,None,None,None,None,1898,1898,Staroměstští hasiči
1,Smích a pláč (395110),Rakousko-Uhersko,1898,1898,premiéra 07/1898 (Výstava architektury a inže...,1.0,[Jan Kříženecký (127419)],film,anekdota,"[hraný, distribuční]",...,None,None,None,None,None,None,None,1898,1898,Smích a pláč
2,Cyklisté (401705),Rakousko-Uhersko,1898,1898,premiéra léto 1898 (Výstava architektury a in...,1.0,[Jan Kříženecký (127419)],film,reportáž,"[dokumentární, distribuční]",...,None,None,None,None,None,None,None,1898,1898,Cyklisté
3,Defilování vojska o Božím těle na Královských ...,Rakousko-Uhersko,1898,1898,premiéra 19. 6. 1898 (Výstava architektury a ...,1.0,[Jan Kříženecký (127419)],film,reportáž,"[dokumentární, distribuční]",...,None,None,None,None,None,None,None,1898,1898,Defilování vojska o Božím těle na Královských ...
4,Hanácké banderium (401707),Rakousko-Uhersko,1898,1898,premiéra 24. 7. 1898 (Výstava architektury a ...,1.0,[Jan Kříženecký (127419)],film,reportáž,"[dokumentární, distribuční]",...,None,None,None,None,None,None,None,1898,1898,Hanácké banderium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4568,Řekni to psem (402288),Česká republika,2022,2021,slavnostní premiéra 29. 6. 2022 (kino Lucerna...,85.0,[Robert Sedláček (493)],film,"[komedie, romantický]","[hraný, distribuční]",...,None,None,None,None,None,None,None,2021,2021,Řekni to psem
4569,A pak přišla láska... (402290),Česká republika,2022,2022,festivalová premiéra 2. 7. 2022 (56. mezináro...,88.0,[Šimon Holý (87211)],film,"[drama, psychologický]","[hraný, distribuční]",...,None,None,None,None,None,None,None,2022,2022,A pak přišla láska...
4570,Stínohra (402264),"[Česká republika, Slovensko]",2022,2020—2021,slavnostní premiéra 5. 4. 2022 (multiplex Cin...,102.0,[Peter Bebjak (10077)],film,"[drama, kriminální, psychologický, thriller]","[hraný, distribuční]",...,None,None,None,None,None,None,None,2020,2021,Stínohra
4571,Idiot (402230),Česká republika,2022,2021,"předpremiéra 2. 3. 2022 (Stavovské divadlo, P...",91.0,[Daniela Špinar (1700)],film,"[drama, záznam divadelní inscenace]","[hraný, distribuční]",...,None,None,None,None,None,None,None,2021,2021,Idiot


In [1110]:
filmy = filmy.sort_values(by='Copyright',ascending=True)

In [1111]:
posledni_filmy = pd.DataFrame(filmy.groupby('film_bez_id')['Film'].last()).reset_index()
posledni_filmy

,film_bez_id,Film
0,"...A bílá bledla, až celkem ztmavla","...A bílá bledla, až celkem ztmavla (397394)"
1,...a bude hůř,...a bude hůř (45816)
2,...a odpusť jim jejich viny,...a odpusť jim jejich viny (397708)
3,...a pozdravuji vlaštovky,...a pozdravuji vlaštovky (396886)
4,...a zase ta Lucie!,...a zase ta Lucie! (397427)
...,...,...
4440,"„Marečku, podejte mi pero!“","„Marečku, podejte mi pero!“ (397112)"
4441,"„Pane, vy jste vdova!“","„Pane, vy jste vdova!“ (396881)"
4442,„Rakev ve snu viděti...“,„Rakev ve snu viděti...“ (396781)
4443,„Zavřeno pro rodinný smutek“,„Zavřeno pro rodinný smutek“ (397686)


In [1112]:
df = df.merge(posledni_filmy, left_on='film_bez_id', right_on='film_bez_id', how='outer')

In [1113]:
len(df)

6619

In [1114]:
df = df.drop(columns=['film_bez_id','index','Id','id','jmeno_bez_id'])

In [1115]:
len(df)

6619

In [1116]:
df = df[~df['rok'].isnull()]

In [1117]:
len(df)

2519

In [1118]:
def naprav_skody(radek):
    if radek['kdo'] == 'nan (nan)':
        kdo = radek['jmeno']
    else:
        kdo = radek['kdo']
    return kdo

In [1119]:
df['kdo'] = df.apply(lambda row: naprav_skody(row), axis = 1)

In [1120]:
df

,rok,datum,kategorie,kdo,film,co,jmeno,Datum narození,Datum úmrtí,Film
0,1993.0,1994-02-25,Nejlepší film,Jiří Ježek (4473),Šakalí léta,vyhra,Jiří Ježek,1950-06-01T00:00:00.000,None,Šakalí léta (7307)
1,1993.0,1994-02-25,Nejlepší režie,Jan Hřebejk (4831),Šakalí léta,vyhra,Jan Hřebejk,1967-06-27T00:00:00.000,None,Šakalí léta (7307)
2,1993.0,1994-02-25,Hlavní mužský herecký výkon,Josef Abrhám (2274),Šakalí léta,vyhra,Josef Abrhám,1939-12-14T00:00:00.000,2022-05-16T00:00:00.000,Šakalí léta (7307)
3,1993.0,1994-02-25,Nejlepší hudba,Ivan Hlas (8715),Šakalí léta,vyhra,Ivan Hlas,1954-05-10T00:00:00.000,None,Šakalí léta (7307)
4,1997.0,1998-02-28,Nejlepší film,Jiří Ježek (4473),Báječná léta pod psa,nominace,Jiří Ježek,1950-06-01T00:00:00.000,None,Báječná léta pod psa (9462)
...,...,...,...,...,...,...,...,...,...,...
2514,2023.0,2024-03-09,Nejlepší krátký film,Kryštof Burda,Osmý den,nominace,Kryštof Burda,NaN,NaN,NaN
2515,2023.0,2024-03-09,Nejlepší krátký film,Tomáš Pertold,Osmý den,nominace,Tomáš Pertold,NaN,NaN,NaN
2516,2023.0,2024-03-09,Nejlepší krátký film,Petr Pylypčuk,Osmý den,nominace,Petr Pylypčuk,NaN,NaN,NaN
2517,2023.0,2024-03-09,Cena Magnesia za nejlepší studentský film,Petr Pylypčuk,Osmý den,nominace,Petr Pylypčuk,NaN,NaN,NaN


In [1121]:
df[df['film'] == 'Atestace']

,rok,datum,kategorie,kdo,film,co,jmeno,Datum narození,Datum úmrtí,Film
2505,2022.0,2023-03-04,Cena Magnesia za nejlepší studentský film,Jan Hecht (86597),Atestace,nominace,Jan Hecht,1993-02-04T00:00:00.000,None,NaN
2506,2023.0,2024-03-09,Nejlepší krátký film,Jan Hecht (86597),Atestace,nominace,Jan Hecht,1993-02-04T00:00:00.000,None,NaN
2507,2023.0,2024-03-09,Nejlepší krátký film,Marek Dusil,Atestace,nominace,Marek Dusil,NaN,NaN,NaN


## Volejte třikrát sláva

In [1122]:
df.to_json(os.path.join('data','ceske_lvy.json'))